# Daily Reported COVID-19 Deaths By U.S. State

#### Which states are doing a good job of flattening the curve?

I got frustrated with the lack of an easy way to compare the progress of COVID-19 between different U.S. states, so I built this baby web scraper to pull reporting data from Wikipedia and plot the daily death count for each state. By starting a new kernel and running all of the following cells, you should see an interactive line plot at the end, which displays the most recently scraped data. 

I used Plotly because it comes with really crucial tools that allow users to zoom in and out of plots, see tooltips, and isolate features (like the line plot for a particular state). While the point of this project was to see all the states plotted together on the same chart, having 54 lines (including U.S. territories Guam, Northern Mariana Islands, Puerto Rico, and the U.S. Virgin Islands) on the same plot isn't really that useful unless you can filter them out.

At the end of this script, I included a gigantic list of improvements that I think would be useful to add. Hopefully if you clone this script, you can make it better!

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [ ]:
url = 'http://en.wikipedia.org/wiki/Template:2019%E2%80%9320_coronavirus_pandemic_data/United_States_medical_cases'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')
# print (soup.prettify)

In [ ]:
# Pull all of the tables out

tables = soup.find_all("table", attrs={"class": "wikitable"})

In [ ]:
# There are multiple tables on the Wikipedia page, so you have to specify that you're 
# just talking about the last one, which has COVID-19 deaths (i.e. not confirmed cases)

# For each row, pull out all of the cells and turn the row into a list of strings
# Get rid of the "\n" newline business

all_rows = []

for table in tables[1]:
    if len(table) > 1:
        rows = table.find_all('tr')
        
        for row in rows[1:]:
            cells = row.find_all('td')
            data = []
            for i in cells:
                data.append(i.text[:-1])
            
            all_rows.append(data)

# Drop the first row, which has no data
all_rows = all_rows[1:]

# print(all_rows)

In [ ]:
# Change the empty strings into 0's, and turn everything into integers
# Also, deal with the negative number (thank you, Pennsylvania) by 
# using absolute values

newrows = []

for row in all_rows:
    i = []
    for el in row:
        if el == '' or int(el) < 0:
            el = 0
        else:
            el = int(el)
        i.append(el)
    newrows.append(i)

# print(newrows)

# EDIT: 

Oops. As always, a good lesson on really getting to know your data before you try to just plug and chug it.

### Some important caveats from Wikipedia:

"On April 19, 2020, Washington State removed 222 cases that were discovered to be out of state residents.

On April 25, 2020, Colorado removed 29 deaths. All 29 removed deaths were probable deaths which were 
entered as duplicates.

On April 23, 2020, Pennsylvania removed 201 deaths. All 201 removed deaths were probable cases which 
needed further investigation in order to be confirmed."

This explains Pennsylvania's negative value. Probably the best way to deal with that is just to set 
that value to zero.

In [ ]:
# Scrape the column names (i.e. states)

colnames = []

for table in tables[1]:
    if len(table) > 1:
        headers = table.find_all('th')
        
        for h in headers[8:63]:
            state = h.text[:-1]
            colnames.append(state)
            
print(colnames)

In [ ]:
# Append the state names as the first row in the list of rows 
# (these state names will be converted to a table header in the next step)

newrows[0] = colnames

# print(newrows)

In [ ]:
# Make a dataframe out of the list of lists

df = pd.DataFrame(newrows)

print(df)

In [ ]:
# Drop the last 5 rows, which don't have any data
newrows2 = newrows[:-5]

df = pd.DataFrame(newrows2)


# # Drop rows 32, 33, 64, and 65, which don't have any data
# This step will need to be adjusted for every new month of collected data
df = df.drop(index=64) \
        .drop(index=65) \
        .drop(index=32) \
        .drop(index=33) \

df.index = range(len(df))

print(df)

In [ ]:
# Convert first row into a header

header = df.iloc[0] 
cleandf = df[1:]
cleandf.columns = header

cleandf.index = range(len(cleandf))

print(cleandf)

In [ ]:
# Get all of the headers, in preparation for pulling the dates

raw = []

for table in tables[1]:
    if len(table) > 1:
        rownames = table.find_all('th')
        
        if len(rownames) > 1:
        
            for h in rownames:
                rowname = h.text[:-1]
                raw.append(rowname)
                
print (raw)

In [ ]:
# Define a function to cut out a slice of elements in the middle of a list

def excise_elements(listname, start_index, num_elements):
        """
        Remove element at `idx` from `seq`.
        TODO: error checks.
        """
        return listname[:start_index] + listname[(start_index + num_elements):]

excise_elements([3, 4, 54, 8, 96, 2], 2, 2)

In [ ]:
# Clean out everything that isn't a date

# Transfer the raw list into a fresh copy
newdates = raw

# Slice off the leading list elements, up to March 1
newdates = raw[69:]

# Slice off the trailing list elements, down to the most recent reporting date
newdates = newdates[:-181]

# !This middle slicing step might change each month
newdates = excise_elements(newdates, 124, 65)  # between March and April
newdates = excise_elements(newdates, 244, 65)     # between April and May
            
print (newdates)
print (len(newdates))

In [ ]:
cleandates = newdates[1::4]

print (cleandates)
print (len(cleandates))

In [ ]:
# Add a column into the dataframe that is populated with the scraped dates

pd.options.mode.chained_assignment = None  # default='warn'

fulldf = cleandf

fulldf['Date'] = cleandates

fulldf.tail(10)

In [ ]:
# Convert the date column into an index

fulldf = fulldf.set_index('Date')
fulldf.tail(10)

In [ ]:
# Plot

fig = go.Figure()

fig.update_layout(
    title={
        'text':"Daily Reported COVID-19 Deaths By State",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title="Date",
    yaxis_title="Number of Reported COVID-19 Deaths",
    template = "plotly_dark"
)

dates = list(fulldf.index.values) 
columns = list(fulldf) 
  
for col in columns: 
    fig.add_trace(go.Scatter(
        x=dates, 
        y=fulldf[col],
        name=col, 
        hovertemplate="Died: %{y} (%{x})", 
        line = dict(shape = 'spline', width= 1.5)
    ))

fig.show()

# Next steps:

- Show smoother spline lines (maybe by calculating a 7-day running average?). It would be nice to see a version of this plot which ignores the near-universal reporting troughs on Sundays and Mondays.


- If not the above point, then shade weekends so that you can check differences in reporting depending on day of the week


- Make the tooltip have an opaque, or more visible, background for the state name


- Sort the elements of the legend in descending order, in relation to number of deaths


-  Pull data automatically on a schedule


- Filter out the plot to show only the "top 20", i.e. worst states for COVID-19 deaths


- Compare confirmed cases to deaths


- Create more plots which compare COVID-19 deaths with other causes of death (flu, car accidents, suicide, heart disease, diabetes)

# Daily Confirmed Cases By State

### Using data reported by NY Times
https://github.com/nytimes/covid-19-data

I wasn't aware of this dataset when I made the web scraper, or I would have used this instead! Much easier. Just cloning or pulling the github repo where the data are being continuously updated, and then importing the csv of states data.

In [1]:
import git 
import pandas as pd
import plotly.graph_objects as go

In [2]:
# Do a git pull to get the latest data

repo_dir = '/Users/evelyn/Workspaces/misc-projects/covid-19-data'
g = git.cmd.Git(repo_dir)
g.pull()

'Already up-to-date.'

In [3]:
rawdf = pd.read_csv("/Users/evelyn/Workspaces/misc-projects/covid-19-data/us-states.csv")
rawdf.tail()

,date,state,fips,cases,deaths
3309,2020-05-01,Virginia,51,16901,581
3310,2020-05-01,Washington,53,15010,829
3311,2020-05-01,West Virginia,54,1151,47
3312,2020-05-01,Wisconsin,55,7314,327
3313,2020-05-01,Wyoming,56,420,7


In [4]:
# Daily cases (cumulative):
casesdf = rawdf.pivot(index='date', columns='state', values='cases').reset_index()

# Filter to only cases after March 13, 2020 - national lockdown
casesdf = casesdf[casesdf['date'] >= '2020-03-13']

casesdf.set_index(['date'], inplace=True)

# casesdf.tail()

In [6]:
# Calculate only new cases each day, not cumulative

newcasesdf = pd.DataFrame()

for state in casesdf:
    newcasesdf[state] = pd.Series(casesdf[state]).diff()

# newcasesdf

In [7]:
# Daily deaths (cumulative):
deathsdf = rawdf.pivot_table(index='date', columns='state', values='deaths').reset_index()

# Filter to only deaths after March 13, 2020 - national lockdown
deathsdf = deathsdf[deathsdf['date'] >= '2020-03-13']

deathsdf.set_index(['date'], inplace=True)

# deathsdf.head()

In [8]:
# Calculate only new deaths each day, not cumulative

newdeathsdf = pd.DataFrame()

for state in deathsdf:
    newdeathsdf[state] = pd.Series(deathsdf[state]).diff()

# newdeathsdf

In [21]:
# Plot daily number of newly confirmed cases by state

fig = go.Figure()

fig.update_layout(
    title={
        'text':"Daily New Confirmed COVID-19 Cases By State",
        'y':0.9,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title="Date",
    yaxis_title="Count of New Cases",
    template = "plotly_dark"
)

dates = list(newcasesdf.index.values) 
columns = list(newcasesdf) 
  
for col in columns: 
    fig.add_trace(go.Scatter(
        x=dates, 
        y=newcasesdf[col],
        name=col, 
        hovertemplate="New Cases: %{y} (%{x})", 
        line = dict(shape = 'spline', width= 1.5)
    ))

fig.show()

In [20]:
# Plot number of newly confirmed deaths by state

fig = go.Figure()

fig.update_layout(
    title={
        'text':"New Daily COVID-19 Deaths By State",
        'y':0.9,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title="Date",
    yaxis_title="Number of Deaths",
    template = "plotly_dark"
)

dates = list(newdeathsdf.index.values) 
columns = list(newdeathsdf) 
  
for col in columns: 
    fig.add_trace(go.Scatter(
        x=dates, 
        y=newdeathsdf[col],
        name=col, 
        hovertemplate="Died: %{y} (%{x})", 
        line = dict(shape = 'spline', width= 1.5)
    ))

fig.show()

In [22]:
# Plot cumulative number of confirmed cases by state

fig = go.Figure()

fig.update_layout(
    title={
        'text':"Total Confirmed COVID-19 Cases By State (Cumulative)",
        'y':0.9,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title="Date",
    yaxis_title="Cumulative Count of Cases",
    template = "plotly_dark"
)

dates = list(casesdf.index.values) 
columns = list(casesdf) 
  
for col in columns: 
    fig.add_trace(go.Scatter(
        x=dates, 
        y=casesdf[col],
        name=col, 
        hovertemplate="Total Cases: %{y} (%{x})", 
        line = dict(shape = 'spline', width= 1.5)
    ))

fig.show()

In [23]:
# Plot reported deaths

fig = go.Figure()

fig.update_layout(
    title={
        'text':"Total Reported COVID-19 Deaths By State (Cumulative)",
        'y':0.9,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title="Date",
    yaxis_title="Cumulative Count of Deaths",
    template = "plotly_dark"
)

dates = list(deathsdf.index.values) 
columns = list(deathsdf) 
  
for col in columns: 
    fig.add_trace(go.Scatter(
        x=dates, 
        y=deathsdf[col],
        name=col, 
        hovertemplate="Total deaths: %{y} (%{x})", 
        line = dict(shape = 'spline', width= 1.5)
    ))

fig.show()